# Computational Theory Assessment: Winter 25/26

In [ ]:
import numpy as np
import math

## Problem 1: Binary Words and Operations

In [ ]:
# Main SHA-256 functions
def Parity(x, y, z):
    """
    Calculate the parity of three 32-bit words.
    
    The parity function returns 1 when an odd number of inputs are 1, 
    and 0 when an even number of inputs are 1. Equivalent to x XOR y XOR z.
    
    Parameters:
        x (int): First 32-bit integer
        y (int): Second 32-bit integer  
        z (int): Third 32-bit integer
        
    Returns:
        numpy.uint32: x XOR y XOR z
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    return x ^ y ^ z

def Ch(x, y, z):
    """
    Choose between y and z based on value of x.
    
    For each bit position, chooses y if x is 1, z if x is 0.
    Equivalent to (x AND y) XOR (NOT x AND z).
    
    Parameters:
        x (int): 32-bit integer used as selector
        y (int): 32-bit integer chosen when x=1
        z (int): 32-bit integer chosen when x=0
        
    Returns:
        numpy.uint32: (x AND y) XOR (NOT x AND z)
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    return (x & y) ^ (~x & z)

def Maj(x, y, z):
    """
    Calculate majority value at each bit position.
    
    For each bit position, returns 1 if at least two inputs are 1.
    Equivalent to (x AND y) XOR (x AND z) XOR (y AND z).
    
    Parameters:
        x (int): First 32-bit integer
        y (int): Second 32-bit integer
        z (int): Third 32-bit integer
        
    Returns:
        numpy.uint32: Majority value at each bit position
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    return (x & y) ^ (x & z) ^ (y & z)

In [ ]:
def Sigma0(x):
    """
    SHA-256 uppercase Sigma0 function.
    
    Performs three bitwise rotations and XORs the results.
    Equivalent to ROTR-2(x) XOR ROTR-13(x) XOR ROTR-22(x).
    
    Parameters:
        x (int): 32-bit integer input
        
    Returns:
        numpy.uint32: Result of the three rotations XORed together
    """
    x = np.uint32(x)
    # Inline rotation implementation
    rotr_2 = (x >> 2) | (x << (32 - 2))
    rotr_13 = (x >> 13) | (x << (32 - 13))
    rotr_22 = (x >> 22) | (x << (32 - 22))
    return rotr_2 ^ rotr_13 ^ rotr_22

def Sigma1(x):
    """
    SHA-256 uppercase Sigma1 function.
    
    Performs three bitwise rotations and XORs the results.
    Equivalent to ROTR-6(x) XOR ROTR-11(x) XOR ROTR-25(x).
    
    Parameters:
        x (int): 32-bit integer input
        
    Returns:
        numpy.uint32: Result of the three rotations XORed together
    """
    x = np.uint32(x)
    # Inline rotation implementation
    rotr_6 = (x >> 6) | (x << (32 - 6))
    rotr_11 = (x >> 11) | (x << (32 - 11))
    rotr_25 = (x >> 25) | (x << (32 - 25))
    return rotr_6 ^ rotr_11 ^ rotr_25

def sigma0(x):
    """
    SHA-256 lowercase sigma0 function.
    
    Performs two rotations and one shift, then XORs the results.
    Equivalent to ROTR-7(x) XOR ROTR-18(x) XOR SHR-3(x).
    
    Parameters:
        x (int): 32-bit integer input
        
    Returns:
        numpy.uint32: Result of the three operations XORed together
    """
    x = np.uint32(x)
    # Inline implementation
    rotr_7 = (x >> 7) | (x << (32 - 7))
    rotr_18 = (x >> 18) | (x << (32 - 18))
    shr_3 = x >> 3
    return rotr_7 ^ rotr_18 ^ shr_3

def sigma1(x):
    """
    SHA-256 lowercase sigma1 function.
    
    Performs two rotations and one shift, then XORs the results.
    Equivalent to ROTR-17(x) XOR ROTR-19(x) XOR SHR-10(x).
    
    Parameters:
        x (int): 32-bit integer input
        
    Returns:
        numpy.uint32: Result of the three operations XORed together
    """
    x = np.uint32(x)
    # Inline implementation
    rotr_17 = (x >> 17) | (x << (32 - 17))
    rotr_19 = (x >> 19) | (x << (32 - 19))
    shr_10 = x >> 10
    return rotr_17 ^ rotr_19 ^ shr_10

In [ ]:
"""
## Testing and Verification

We test all 7 required functions with representative 32-bit values 
to verify correctness according to the SHA-256 specification.
"""

print("=== Testing Required SHA-256 Functions ===\n")

print("1. Testing Parity, Ch, and Maj functions:")
print(f"Parity(0x00000001, 0x00000001, 0x00000000) = {Parity(0x00000001, 0x00000001, 0x00000000):08x}")
print(f"Ch(0xFFFFFFFF, 0x12345678, 0x87654321) = {Ch(0xFFFFFFFF, 0x12345678, 0x87654321):08x}")
print(f"Maj(0xF0F0F0F0, 0xFF00FF00, 0x0F0F0F0F) = {Maj(0xF0F0F0F0, 0xFF00FF00, 0x0F0F0F0F):08x}")
print()

print("2. Testing Sigma functions:")
test_val = 0x12345678
print(f"Sigma0(0x12345678) = {Sigma0(test_val):08x}")
print(f"Sigma1(0x12345678) = {Sigma1(test_val):08x}")
print(f"sigma0(0x12345678) = {sigma0(test_val):08x}")
print(f"sigma1(0x12345678) = {sigma1(test_val):08x}")

## Problem 2: Fractional Parts of Cube Roots

In [ ]:
def primes(n):
    """
    Generate the first n prime numbers using the Sieve of Eratosthenes.
    
    Parameters:
        n (int): Number of prime numbers to generate
        
    Returns:
        list: First n prime numbers in ascending order
        
    Raises:
        ValueError: If n is not a positive integer
    """
    if not isinstance(n, int) or n <= 0:
        raise ValueError("n must be a positive integer")
    
    if n == 1:
        return [2]
    
    # Estimate upper bound for the nth prime
    if n < 6:
        upper_bound = 20
    else:
        upper_bound = int(n * (math.log(n) + math.log(math.log(n)))) + 10
    
    sieve = [True] * (upper_bound + 1)
    sieve[0:2] = [False, False]
    
    primes_list = []
    for num in range(2, upper_bound + 1):
        if sieve[num]:
            primes_list.append(num)
            if len(primes_list) == n:
                break
            for multiple in range(num * num, upper_bound + 1, num):
                sieve[multiple] = False
    
    return primes_list

# Test prime number generation
print("=== Testing Prime Number Generator ===\n")

test_cases = [1, 5, 10, 64]
for n in test_cases:
    prime_list = primes(n)
    print(f"First {n} primes: {prime_list}")
    print(f"Length: {len(prime_list)}, Last prime: {prime_list[-1]}\n")

## Problem 3: Padding

# Problem 4: Hashes

# Problem 5: Passwords